# Total points 8

# In  the homework
- We'll collect climate change related tweet from twitter using REST API
- Use tweets to populate Pandas DataFrames
- Perform some elementary pandas operation to check data sanity


**Please read**

- https://developer.twitter.com/en/developer-terms/more-on-restricted-use-cases


Before proceeding with homework.

<font size="6" color='red'> Create new app at https://apps.twitter.com/app/new  (You may have to create twitter account before this) </font>





[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


Once you have create app, click on app details and keys and token tab

<font size="6" color='red'> write API key value for client_key and API secret key value for client_secret in the next cell</font>

In [5]:
client_key = "SjJ1AVttBX9DLEDTGPS1xaxCC"
client_secret = "iMe9fK5YweWMxyI7OqeXLEAAaQcWjBjWKWGhRvTQEnLJOGZyVq"

In [6]:
import requests
import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [7]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [8]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [9]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAB6MegEAAAAAELAHQLkbwYGYFLLCZcvnmPGHL8U%3DRgitE4Vpk7yjFGT4RZ6tZ7Kjry8qolUK4w4zbdvrQAtIQhj0V8'}

In [10]:
access_token = json_data['access_token']

# Let's do some search on climate change
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [11]:
max_tweets=100
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []
users =[]

In [12]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

parameters = { 'query': 'climate change',
                'max_results': 100,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'}

search_url = base_url+'2/tweets/search/recent'

response = requests.get(search_url, headers=search_headers, params=parameters)

In [13]:
climate_tweets = response.json()


In [14]:
len(climate_tweets['data'])

100

In [15]:
tweets +=climate_tweets['data']


In [16]:

 ids = [tw['id'] for tw in climate_tweets['data']]
    

In [17]:
min_id = min(ids)

In [18]:
tw_ids = []
tw_ids+=ids

As per twitter api documentation we can use

until_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [19]:
 parameters = { 'query': 'climate change',
                'max_results': tw_limit,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',
                "since_id": min_id
                 }

for  i in range(max_tweets//tw_limit -1):
    print("searching tweets with id  < {}".format(min_id))
    search_url = base_url+'2/tweets/search/recent'
    response = requests.get(search_url, headers=search_headers, params=parameters)
    climate_tweets = response.json()
    ids = [tw['id'] for tw in climate_tweets['statuses']]
    tw_ids+=ids
    print(i, len(climate_tweets['data']))
    min_id = min(ids)
    tweets +=climate_tweets['data']
    users.extend(climate_tweets['includes']['users'])
    if 'next_token' in response.json()['meta']:
					parameters['next_token'] = response.json()['meta']['next_token']

In [20]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 100


# Let's print some tweets

In [21]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [22]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'author_id': '1489372213102972932',
  'conversation_id': '1551299627974103041',
  'created_at': '2022-07-24T20:13:47.000Z',
  'entities': {'mentions': [{'end': 16,
                             'id': '707231479047315456',
                             'start': 3,
                             'username': 'PeterSweden7'}]},
  'id': '1551299627974103041',
  'lang': 'en',
  'public_metrics': {'like_count': 0,
                     'quote_count': 0,
                     'reply_count': 0,
                     'retweet_count': 369},
  'referenced_tweets': [{'id': '1551292683376660480', 'type': 'retweeted'}],
  'reply_settings': 'everyone',
  'source': 'Twitter for iPhone',
  'text': 'RT @PeterSweden7: The same people telling you to eat bugs are '
          'themselves dining on finest meat.\n'
          '\n'
          'The same people telling you to get rid of…'},
 {'author_id': '463325504',
  'conversation_id': '1551299627571462145',
  'created_at': '2022-07-24T20:13:47.000Z',
  'entities': {

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [23]:
for status in tweets[0:max_tweet_to_print]:
    print(status['text'] + '\n')

RT @PeterSweden7: The same people telling you to eat bugs are themselves dining on finest meat.

The same people telling you to get rid of…

RT @newsmax: House Minority Leader Kevin McCarthy rejected calls for President Joe Biden to declare a climate change emergency.  https://t.…

RT @AnnaWilliams41: @TrophyXpose @SafariClubIntl @tarastrong @AgainstHunting @SARA2001NOOR @Angelux1111 @ladydobe1 @Lin11W @Protect_Wldlife…

@GOP Womens rights crisis? Threatening Ukraine crisis? Overthrowing the government crisis? Bailing out of climate change crisis? Gay marriage crisis? Supporting Russia over US crisis? Which one



# Pandas DataFrame for storing tweet feed

Create following date frame for storing tweets

- **climate_feed_df** DataFrame for storing tweet
- **user_df** DataFrame for storing user detail
- **retweeted_status_df** for storing original tweet

# Printing keys in tweets

In [24]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['entities', 'public_metrics', 'id', 'lang', 'referenced_tweets', 'source', 'created_at', 'conversation_id', 'author_id', 'text', 'reply_settings'])
11


In [25]:
import pandas as pd
import copy

# Generators for building DataFrames

# Q2 (2 point) Complete following   python generator *tweet_generator*. Generator yield modified tweet .
For each tweet in tweets
- Replace **author_id** attribute with **user_id**.
- Replace **referenced_tweets** attribute with **referenced_tweets_ids** and value(s) with **tweet['referenced_tweets']['id']** value(s).
- Keep other attributes in the tweet as such.

In [100]:
def tweet_generator(tweets):
    for tweet in tweets:
        modified_tweet= copy.deepcopy(tweet)
        # remove author_id and store the value in user_id
        modified_tweet["user_id"] = modified_tweet.pop("author_id")
        if "referenced_tweets" in modified_tweet:
            referenced_tweets = modified_tweet.pop("referenced_tweets")
        else:
            referenced_tweets = []
        modified_tweet["referenced_tweets_ids"] = list(map(lambda x: x['id'], referenced_tweets))
        if "public_metrics" in modified_tweet and "retweet_count" in  modified_tweet["public_metrics"]:
            modified_tweet["retweet_count"] = modified_tweet["public_metrics"]["retweet_count"]
        else:
            modified_tweet["retweet_count"] = len(modified_tweet["referenced_tweets_ids"])
        #Write your code here
        
        
        yield modified_tweet       
            
            

In [35]:
#def tweet_generator(tweets):
#    for tweet in tweets:
#        try:
#            modified_tweet = copy.deepcopy(tweet)
#            modified_tweet['user_id'] = modified_tweet.pop('author_id')
#            modified_tweet['referenced_tweets_ids'] = modified_tweet['referenced_tweets'][0]['id']
#            modified_tweet['retweet_count'] = modified_tweet['public_metrics']['retweet_count']
#            yield modified_tweet
#        except KeyError as e:
#            continue

In [101]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'public_metrics': {'retweet_count': 369,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'id': '1551299627974103041',
  'lang': 'en',
  'source': 'Twitter for iPhone',
  'created_at': '2022-07-24T20:13:47.000Z',
  'entities': {'mentions': [{'start': 3,
     'end': 16,
     'username': 'PeterSweden7',
     'id': '707231479047315456'}]},
  'conversation_id': '1551299627974103041',
  'text': 'RT @PeterSweden7: The same people telling you to eat bugs are themselves dining on finest meat.\n\nThe same people telling you to get rid of…',
  'reply_settings': 'everyone',
  'user_id': '1489372213102972932',
  'referenced_tweets_ids': ['1551292683376660480'],
  'retweet_count': 369},
 {'entities': {'annotations': [{'start': 35,
     'end': 48,
     'probability': 0.9745,
     'type': 'Person',
     'normalized_text': 'Kevin McCarthy'},
    {'start': 79,
     'end': 87,
     'probability': 0.6566,
     'type': 'Person',
     'normalized_text': 'Joe Biden'}],
   'mentions': [{'st

In [102]:
climate_feed_df = pd.DataFrame(tweet_generator(tweets))

In [103]:
climate_feed_df

,public_metrics,id,lang,source,created_at,entities,conversation_id,text,reply_settings,user_id,referenced_tweets_ids,retweet_count,in_reply_to_user_id,withheld
0,"{'retweet_count': 369, 'reply_count': 0, 'like...",1551299627974103041,en,Twitter for iPhone,2022-07-24T20:13:47.000Z,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1551299627974103041,RT @PeterSweden7: The same people telling you ...,everyone,1489372213102972932,[1551292683376660480],369,NaN,NaN
1,"{'retweet_count': 50, 'reply_count': 0, 'like_...",1551299627571462145,en,Twitter for Android,2022-07-24T20:13:47.000Z,"{'annotations': [{'start': 35, 'end': 48, 'pro...",1551299627571462145,RT @newsmax: House Minority Leader Kevin McCar...,everyone,463325504,[1551292388601057280],50,NaN,NaN
2,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",1551299627189768193,en,Twitter for Android,2022-07-24T20:13:47.000Z,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",1551299627189768193,RT @AnnaWilliams41: @TrophyXpose @SafariClubIn...,everyone,3366336160,[1551211702120841222],2,NaN,NaN
3,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1551299626598387713,en,Twitter for iPhone,2022-07-24T20:13:47.000Z,"{'annotations': [{'start': 39, 'end': 45, 'pro...",1551159256103141376,@GOP Womens rights crisis? Threatening Ukraine...,everyone,1408706413984108544,[1551159256103141376],0,11134252,NaN
4,"{'retweet_count': 655, 'reply_count': 0, 'like...",1551299622026354690,en,Twitter for iPhone,2022-07-24T20:13:46.000Z,"{'annotations': [{'start': 20, 'end': 22, 'pro...",1551299622026354690,"RT @jayrosen_nyu: A BBC story about ""The audac...",everyone,141977378,[1551221968258895873],655,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1551299364508897280,en,Twitter for Android,2022-07-24T20:12:44.000Z,"{'urls': [{'start': 188, 'end': 211, 'url': 'h...",1551299364508897280,Climate change learn what real scientists have...,everyone,95025717,[],0,NaN,NaN
96,"{'retweet_count': 369, 'reply_count': 0, 'like...",1551299364362010624,en,Twitter for iPhone,2022-07-24T20:12:44.000Z,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1551299364362010624,RT @PeterSweden7: The same people telling you ...,everyone,3176841441,[1551292683376660480],369,NaN,NaN
97,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1551299362860531721,en,Twitter Web App,2022-07-24T20:12:44.000Z,"{'mentions': [{'start': 0, 'end': 11, 'usernam...",1550986861987913728,@kotosquito @sotnasoinotna And more. Too many ...,everyone,1463278009247354882,[1551296034449637377],0,836576099370995712,NaN
98,"{'retweet_count': 369, 'reply_count': 0, 'like...",1551299358976577539,en,Twitter for Android,2022-07-24T20:12:43.000Z,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1551299358976577539,RT @PeterSweden7: The same people telling you ...,everyone,824036799140036609,[1551292683376660480],369,NaN,NaN


# Q3(1 points) List datatype using climate_feed_df. Describe which attributes doesn't have correct data types. What should be correct datatype?

In [104]:
# write code here datatype listing
climate_feed_df.dtypes

public_metrics           object
id                       object
lang                     object
source                   object
created_at               object
entities                 object
conversation_id          object
text                     object
reply_settings           object
user_id                  object
referenced_tweets_ids    object
retweet_count             int64
in_reply_to_user_id      object
withheld                 object
dtype: object

# Write columns with wrong datatype and mention correct data type too.

*double click this cell to edit*
- created_at , should be datetime

all other fields are 'string' in the API specification, which maps to pandas 'object'

# Q4 (1.5 point) Complete following tweet_user_generator. Generator should yield only *user* value from a tweet

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [90]:
# write code here
def tweet_user_generator(tweets):
    df = None
    queried_users = []
    users = []
    for t in tweets["data"]:
        user_id = t["author_id"]
        if user_id not in queried_users:
            user_url = f'{base_url}2/users/{user_id}'
#            print(f'calling {user_url}')
            queried_users.append(user_id)
            response = requests.get(user_url, headers=search_headers)
            print(f'status code : {response.status_code}')
            if response.status_code == 200:
                users.append(response.json()["data"])
    return users

In [105]:
user_df = pd.DataFrame(tweet_user_generator(climate_tweets))

status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status code : 200
status cod

In [106]:
user_df

,id,name,username,withheld
0,1489372213102972932,TheTexasTrowIV,TheTexasTrow4,NaN
1,463325504,Janet Lampron,JanetLampron,NaN
2,3366336160,Geri Marie,ladydobe1,NaN
3,1408706413984108544,Toni,ToniMax93160066,NaN
4,141977378,Graham Brown. 😜 🍁 💬💉,SilvertipGrizz,NaN
...,...,...,...,...
92,443806263,Anthony Rose,rosea_tony,NaN
93,1370046545903689729,Sime,Sime50068528,NaN
94,191096018,Sylvia Pellicore,Roenok,NaN
95,95025717,Bob Perella,bobperella,NaN


# Q5(1 point) List datatypes of columns using user_df. Describe which attributes doesn't have correct data types. What should be correct datatype for each case?

In [107]:
# write code here for datatype listing
user_df.dtypes



id          object
name        object
username    object
withheld    object
dtype: object

# write columns with wrong datatype and mention correct data type too.

*double click this cell to edit*
All the fields I have look to be correct

# Q6 (1 point) Complete following retweeted_status_generator. Generator should yield dictionary of retweeted_status value from each tweet. If a tweet doesn't have this key return empty dictionary.

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [108]:
retweeted_status_df = climate_feed_df[['id', 'retweet_count']]

In [109]:
retweeted_status_df.head()

,id,retweet_count
0,1551299627974103041,369
1,1551299627571462145,50
2,1551299627189768193,2
3,1551299626598387713,0
4,1551299622026354690,655


# Q7 (1 point) Write code to find number of rows in retweeted_status_df that were not retweeted. print number of such rows.

In [111]:
# Write code here
retweeted_status_df[retweeted_status_df["retweet_count"] == 0].shape[0]

31